<a href="https://colab.research.google.com/github/DrAlexSanz/Electricity-prices/blob/main/Time_series_stationality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
from google.colab import drive

In [3]:
drive.mount('/gdrive')

data = pd.read_csv('/gdrive/MyDrive/Electricity_prices/full_data.csv')

MessageError: ignored

## Several plots to check how the price looks like and the power generation too


*   Create a plotting function, give the df, the name of the column and it will generate the plot
*   Explore several plots.



In [ ]:
def plot_column(df, column_name,
                label = None, ylabel = None, title = None, start=0, end = None):
    """
    Plots a column of the weather dataframe. The user can define either the 'df' and the 'column' 
    or the 'series' and additionally, can also define the 'label', the 
    'ylabel', the 'title', the 'start' and the 'end' of the plot.
    """
    sns.set()
    fig, ax = plt.subplots(figsize=(30, 12))
    ax.set_xlabel('Time', fontsize=16)
    if column:
        ax.plot(df[column_name][start:end], label=label)
        ax.set_ylabel(ylabel, fontsize=16)
    if ylabel:
        ax.legend(fontsize=16)
    if title:
        ax.set_title(title, fontsize=24)
    ax.grid(True)
    return ax

,time,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind onshore,total load forecast,total load actual,price actual,temp_ Barcelona,pressure_ Barcelona,humidity_ Barcelona,wind_speed_ Barcelona,wind_deg_ Barcelona,rain_1h_ Barcelona,snow_3h_ Barcelona,clouds_all_ Barcelona,temp_Bilbao,pressure_Bilbao,humidity_Bilbao,wind_speed_Bilbao,wind_deg_Bilbao,rain_1h_Bilbao,snow_3h_Bilbao,clouds_all_Bilbao,temp_Madrid,pressure_Madrid,humidity_Madrid,wind_speed_Madrid,wind_deg_Madrid,rain_1h_Madrid,snow_3h_Madrid,clouds_all_Madrid,temp_Seville,pressure_Seville,humidity_Seville,wind_speed_Seville,wind_deg_Seville,rain_1h_Seville,snow_3h_Seville,clouds_all_Seville,temp_Valencia,pressure_Valencia,humidity_Valencia,wind_speed_Valencia,wind_deg_Valencia,rain_1h_Valencia,snow_3h_Valencia,clouds_all_Valencia
0,2014-12-31 23:00:00+00:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,43.0,73.0,49.0,196.0,6378.0,26118.0,25385.0,65.41,281.625,1035.0,100,7.0,58,0.0,0.0,0,269.657312,1036.0,97,0.0,226,0.0,0.0,0,267.325,971.0,63,1.0,309,0.0,0.0,0,273.375,1039.0,75,1.0,21,0.0,0.0,0,270.475,1001.0,77,1.0,62,0.0,0.0,0
1,2015-01-01 00:00:00+00:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,43.0,71.0,50.0,195.0,5890.0,24934.0,24382.0,64.92,281.625,1035.0,100,7.0,58,0.0,0.0,0,269.763500,1035.0,97,0.0,229,0.0,0.0,0,267.325,971.0,63,1.0,309,0.0,0.0,0,273.375,1039.0,75,1.0,21,0.0,0.0,0,270.475,1001.0,77,1.0,62,0.0,0.0,0
2,2015-01-01 01:00:00+00:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,43.0,73.0,50.0,196.0,5461.0,23515.0,22734.0,64.48,281.286,1036.0,100,7.0,48,0.0,0.0,0,269.251688,1036.0,97,1.0,224,0.0,0.0,0,266.186,971.0,64,1.0,273,0.0,0.0,0,274.086,1039.0,71,3.0,27,0.0,0.0,0,269.686,1002.0,78,0.0,23,0.0,0.0,0
3,2015-01-01 02:00:00+00:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,43.0,75.0,50.0,191.0,5238.0,22642.0,21286.0,59.32,281.286,1036.0,100,7.0,48,0.0,0.0,0,269.203344,1035.0,97,1.0,225,0.0,0.0,0,266.186,971.0,64,1.0,273,0.0,0.0,0,274.086,1039.0,71,3.0,27,0.0,0.0,0,269.686,1002.0,78,0.0,23,0.0,0.0,0
4,2015-01-01 03:00:00+00:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,43.0,74.0,42.0,189.0,4935.0,21785.0,20264.0,56.04,281.286,1036.0,100,7.0,48,0.0,0.0,0,269.485500,1035.0,97,1.0,221,0.0,0.0,0,266.186,971.0,64,1.0,273,0.0,0.0,0,274.086,1039.0,71,3.0,27,0.0,0.0,0,269.686,1002.0,78,0.0,23,0.0,0.0,0
